In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [223]:
raw_data = pd.ExcelFile('rbull1.xlsx')

In [224]:
raw_data.sheet_names

['rbull1.csv']

In [225]:
data = raw_data.parse('rbull1.csv')

In [226]:
data.head()

,date,id,clicks,conversions,cost,impressions
0,2018-05-01,23842875301030600,2,1,4.360000,227
1,2018-05-01,23842872626260600,1,0,5.520258,263
2,2018-05-01,23842875301030600,5,2,5.340000,278
3,2018-05-01,23842860746290600,0,0,0.070000,1
4,2018-05-01,23842872626260600,5,1,13.080612,702


In [227]:
data['ClicksperCost'] = (data['clicks']/data['cost']).replace([np.inf, -np.inf], 0)

In [228]:
data.head()

,date,id,clicks,conversions,cost,impressions,ClicksperCost
0,2018-05-01,23842875301030600,2,1,4.360000,227,0.458716
1,2018-05-01,23842872626260600,1,0,5.520258,263,0.181151
2,2018-05-01,23842875301030600,5,2,5.340000,278,0.936330
3,2018-05-01,23842860746290600,0,0,0.070000,1,0.000000
4,2018-05-01,23842872626260600,5,1,13.080612,702,0.382245


In [229]:
opt_data = (data[['date','id','ClicksperCost']].copy(deep = True))

In [230]:
data_final = pd.pivot_table(data=opt_data, values= 'ClicksperCost', columns='id',fill_value= 0,index='date',aggfunc='sum').reset_index()
data_final.drop('date',inplace= True, axis=1)

In [231]:
data_initial_set = pd.pivot_table(data = data, values= ['clicks','conversions'], fill_value= 0, index='id',aggfunc='sum')

In [232]:
data_initial_set['TotalClicksperConversion'] = (data_initial_set['clicks']/data_initial_set['conversions']).replace([np.inf, -np.inf,np.NAN], 0)

In [233]:
data_initial_set = pd.pivot_table(data = data_initial_set, columns= 'id', values= 'TotalClicksperConversion', fill_value= 0).reset_index()

In [234]:
data_initial_set.drop('index',inplace= True,axis =1)

In [235]:
data_initial_set

id,6088935321237,6088935321437,6088935333637,6088935334437,6088935350037,6088935350837,6088935363037,6088935363237,6088935365037,6088935365837,...,23843003186980600,23843003186990600,23843003187000600,23843003187010600,23843003187020600,23843003187410600,23843003187420600,23843003187430600,23843003187440600,23843003187450600
0,0.0,5.0,6.5,0.0,2.333333,0.0,0.0,0.0,0.0,2.0,...,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,2.0


In [236]:
import gc
del opt_data
del raw_data
del data
gc.collect()

231638

In [ ]:

len_data_final = data_final.shape[1]
while len_data_final > 1:
    appender = data_final.iloc[[0]]
    df = df.append(appender,ignore_index= True)

    len_data_final -= 1
df.shape 

In [ ]:
df = df.append(data_initial_set)
data_final = df.append(data_final)
data_final.shape

In [ ]:
import math
N = data_final.shape[0]
d = data_final.shape[1]
ads_selected = []
numbers_of_selections = [0] * d
sums_of_rewards = [0] * d
total_reward = 0
for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_rewards[i] / numbers_of_selections[i]
            delta_i = math.sqrt(3/2 * math.log(n + 1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad)
    numbers_of_selections[ad] = numbers_of_selections[ad] + 1
    reward = data_final.values[n, ad]
#     print(data_final.values[n, ad])
    sums_of_rewards[ad] = sums_of_rewards[ad] + reward
    total_reward = total_reward + reward


#Visualising the results
plt.hist(ads_selected)
plt.title('Histogram of ads selections')
plt.xlabel('Ads')
plt.ylabel('Number of times each ad was selected')
plt.show()

In [ ]:
from collections import defaultdict
id_reward_map = defaultdict()
for i, id in enumerate(data_final.columns):
    id_reward_map[id] = [i, sums_of_rewards[i], numbers_of_selections[i]]

In [ ]:
sorted(id_reward_map.items(), key=lambda x:(x[1][2]), reverse = True)

In [220]:
data_final.head()

id,6088935321237,6088935321437,6088935333637,6088935334437,6088935350037,6088935350837,6088935363037,6088935363237,6088935365037,6088935365837,...,23843003186980600,23843003186990600,23843003187000600,23843003187010600,23843003187020600,23843003187410600,23843003187420600,23843003187430600,23843003187440600,23843003187450600
0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
